In [ ]:
import torch.nn as nn
from torchvision import models

# Simple CNN Model

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # 1 input channel, 10 output channel, 5 by 5 kernel size
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)

        # 10 input channel, 20 output channel, 5 by 5 kernel size
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)

        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.conv1(x)
        # Apply max pooling with stride of 2 by 2
        # Define 2 for stride will automatically define a square windows stride
        x = F.max_pool2d(s, 2)
        x = F.relu(x)

        x = self.conv2(x)
        # Apply max pooling with stride of 2 by 2
        x = F.max_pool2d(x, 2)
        x = F.relu(x)

        # Flatten max pooling output to fit fully connected
        # View is the same as .flatten() in numpy
        # -1 means fit any remaining to tensor to have same shape after defining 320
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        
        x = self.fc2(x)
        return F.log_softmax(x)

In [ ]:
model = Net()
model

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

# Transfer Learning CNN model

In [ ]:
model = models.vgg16(pretrained=True)

# Freeze model weights
# By default, all weights are marked as requires_grad = True
# This will enable the weights to be changed
# When freezed, the value of weights will not change during update
for param in model.parameters():
    param.requires_grad = False

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


In [ ]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

This model is trained to predict 1000 classes. To use this for other classification tasks, the number of outputs must match the number of classes.

For example, if you want to do transfer learning to predict cats and dogs, the out_features of classifier[6] must be 2.

In [ ]:
model.classifier[6]

Linear(in_features=4096, out_features=1000, bias=True)

In [ ]:
model.classifier[6].weight

Parameter containing:
tensor([[ 1.3450e-02,  4.2154e-02, -2.3040e-03,  ..., -4.9025e-04,
          1.8880e-02, -1.4209e-02],
        [ 7.6020e-03,  4.7305e-02, -5.0164e-03,  ..., -4.9127e-03,
         -5.6295e-03, -1.4197e-02],
        [ 2.1238e-03, -1.2520e-02, -1.8903e-02,  ..., -1.0263e-02,
          3.0020e-02, -2.8852e-02],
        ...,
        [-9.8278e-03,  3.2054e-02,  3.5979e-02,  ..., -5.6409e-03,
          8.3202e-03, -7.5155e-03],
        [ 1.6646e-02, -1.1247e-03,  1.5044e-03,  ..., -9.1578e-03,
         -8.6418e-03, -2.0923e-02],
        [-6.4900e-06, -2.2274e-02,  5.2750e-04,  ...,  4.4403e-02,
         -9.4047e-03, -1.2332e-02]])

To change the model, you can overite the layer. The in_features must match the output of previous layer. In this case, it is the classifier[3] which is a Linear layer. The output is 4096.

In [ ]:
model.classifier[6] = nn.Linear(4096, 2)

Check classifier[6] layer

In [ ]:
model.classifier[6]

Linear(in_features=4096, out_features=2, bias=True)

Make sure this layer is set to requires_grad=True to train the layer

In [ ]:
model.classifier[6].weight

Parameter containing:
tensor([[ 0.0038, -0.0101,  0.0130,  ...,  0.0015, -0.0095, -0.0104],
        [ 0.0086,  0.0121,  0.0056,  ...,  0.0076, -0.0065, -0.0093]],
       requires_grad=True)

In [ ]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1